<a href="https://colab.research.google.com/github/Andrea227/dailyexercise/blob/master/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# B455 Final Project: COVID-19 New York State New Cases Prediction Project

## Executive Summary

This project is built to predict cumulative case in a paticular of any county of New York state, based on the base information of the county, test information and confirming information in the state.

The original data is credit to the official data provided by New York state government, New York city government and personal collected data by Jieying Wu on Kaggle. 

![photo1.png](https://i.loli.net/2020/05/06/ci5rUMZ8alHnkTv.png)

## 1. Import Data and Form Information Dataset

Here I am going to use four datasets to combine my own dataset for research. The first dataset contains covid 19 cases reported by all US counties; the second dataset contains base information including population, sex proportions, and median ages of each county in US; the New York statewide testing datasets provides positive and testing information; the last datasets provides hospitalization condition in New York city.

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import learning_curve, ShuffleSplit
import matplotlib.pyplot as plt

In [0]:
dat1 = pd.read_csv('https://raw.githubusercontent.com/Andrea227/dailyexercise/master/covid_us_county.csv')
dat2 = pd.read_csv('https://raw.githubusercontent.com/Andrea227/dailyexercise/master/us_county.csv')
dat3 = pd.read_csv('https://raw.githubusercontent.com/Andrea227/dailyexercise/master/New_York_State_Statewide_COVID-19_Testing.csv')
dat4 = pd.read_csv('https://raw.githubusercontent.com/Andrea227/dailyexercise/master/data-4SfjZ%20(1).csv')

In [3]:
dat1.tail()

,fips,county,state,lat,long,date,cases,state_code,deaths
335878,NaN,Weber-Morgan,Utah,41.27116,-111.914512,43950,151,UT,2
335879,NaN,Weber-Morgan,Utah,41.27116,-111.914512,43951,158,UT,2
335880,NaN,Weber-Morgan,Utah,41.27116,-111.914512,43952,162,UT,2
335881,NaN,Weber-Morgan,Utah,41.27116,-111.914512,43953,166,UT,2
335882,NaN,Weber-Morgan,Utah,41.27116,-111.914512,43954,167,UT,2


In [4]:
dat2.head()

,fips,county,state,state_code,male,female,median_age,population,female_percentage,lat,long
0,1001,Autauga County,Alabama,AL,26874,28326,37.8,55200,51.315217,32.534923,-86.642730
1,1003,Baldwin County,Alabama,AL,101188,106919,42.8,208107,51.376936,30.727479,-87.722564
2,1005,Barbour County,Alabama,AL,13697,12085,39.9,25782,46.873788,31.869581,-85.393210
3,1007,Bibb County,Alabama,AL,12152,10375,39.9,22527,46.055844,32.998628,-87.126475
4,1009,Blount County,Alabama,AL,28434,29211,40.8,57645,50.673953,33.980869,-86.567380


In [5]:
dat3.head()

,date,County,New Positives,Cumulative Number of Positives,Total Number of Tests Performed,Cumulative Number of Tests Performed
0,43955,Albany,7,1294,130,12084
1,43955,Allegany,0,35,31,637
2,43955,Bronx,253,39476,2001,103476
3,43955,Broome,4,338,180,3533
4,43955,Cattaraugus,0,53,14,1142


In [6]:
dat4.head()

,date,Hospitalizations
0,43893,7.0
1,43894,8.0
2,43895,13.0
3,43896,12.0
4,43897,10.0


Since we are only going to use data from new york city. We are going to clean the data and reform the four dataset in one dataset by merging on their same keys.

In [0]:
dat1 = dat1[dat1['state_code'] == 'NY']
dat2 = dat2[dat2['state_code'] == 'NY']

In [0]:
datw = pd.merge(dat1, dat2, on = ['fips'])

In [9]:
datw.head()

,fips,county_x,state_x,lat_x,long_x,date,cases,state_code_x,deaths,county_y,state_y,state_code_y,male,female,median_age,population,female_percentage,lat_y,long_y
0,36001.0,Albany,New York,42.600603,-73.977239,43852,0,NY,0,Albany County,New York,NY,148910,158516,37.9,307426,51.562327,42.600177,-73.973555
1,36001.0,Albany,New York,42.600603,-73.977239,43853,0,NY,0,Albany County,New York,NY,148910,158516,37.9,307426,51.562327,42.600177,-73.973555
2,36001.0,Albany,New York,42.600603,-73.977239,43854,0,NY,0,Albany County,New York,NY,148910,158516,37.9,307426,51.562327,42.600177,-73.973555
3,36001.0,Albany,New York,42.600603,-73.977239,43855,0,NY,0,Albany County,New York,NY,148910,158516,37.9,307426,51.562327,42.600177,-73.973555
4,36001.0,Albany,New York,42.600603,-73.977239,43856,0,NY,0,Albany County,New York,NY,148910,158516,37.9,307426,51.562327,42.600177,-73.973555


In [0]:
datw = datw.drop(['lat_x', 'long_x', 'state_y', 'county_y', 'state_x','state_code_x','state_code_y', 'date', 'cases', 'deaths'], axis= 1)

In [0]:
datt = datw.drop_duplicates()
dat3 = dat3.drop_duplicates()

In [0]:
data = pd.merge(datt, dat3, left_on = 'county_x', right_on='County')

In [0]:
datf = data.drop(['county_x','fips'], axis=1)

In [0]:
datff = pd.merge(datf, dat4, on = 'date')
datff = datff.sort_values(by='date',ascending=True)

In [15]:
datff.rename(columns={'Hospitalizations':'NYCumuHospitalizations'})

,male,female,median_age,population,female_percentage,lat_y,long_y,date,County,New Positives,Cumulative Number of Positives,Total Number of Tests Performed,Cumulative Number of Tests Performed,NYCumuHospitalizations
3905,11999,13010,41.3,25009,52.021272,42.633449,-77.105468,43893,Yates,0,0,0,0,7.0
3871,359374,384874,38.9,744248,51.713139,43.146446,-77.696089,43893,Monroe,0,0,0,0,7.0
3870,35252,36107,41.7,71359,50.599084,42.912766,-75.669649,43893,Madison,0,0,0,0,7.0
3869,32050,31857,40.8,63907,49.848999,42.728063,-77.775494,43893,Livingston,0,0,0,0,7.0
3868,13451,13268,42.0,26719,49.657547,43.784657,-75.448849,43893,Lewis,0,0,0,0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,189484,188743,36.9,378227,49.902043,41.402137,-74.305545,43955,Orange,129,9144,869,30648,NaN
34,53704,55768,43.5,109472,50.942707,42.852848,-77.299820,43955,Ontario,1,97,86,2021,NaN
33,223995,240247,39.0,464242,51.750380,43.005814,-76.194635,43955,Onondaga,31,1002,176,12639,NaN
47,15620,15744,45.1,31364,50.197679,42.588217,-74.442114,43955,Schoharie,0,45,29,719,NaN


In [0]:
datff = datff.fillna(method = 'pad')

In [0]:
from google.colab import files
datff.to_csv('final.csv') 
files.download('final.csv')

Now we are able to see the info provided by the final datasets describing the COVID-19 information in New York statewise by county.

In [17]:
datff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3906 entries, 3905 to 0
Data columns (total 14 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   male                                  3906 non-null   int64  
 1   female                                3906 non-null   int64  
 2   median_age                            3906 non-null   float64
 3   population                            3906 non-null   int64  
 4   female_percentage                     3906 non-null   float64
 5   lat_y                                 3906 non-null   float64
 6   long_y                                3906 non-null   float64
 7   date                                  3906 non-null   int64  
 8   County                                3906 non-null   object 
 9   New Positives                         3906 non-null   int64  
 10  Cumulative Number of Positives        3906 non-null   int64  
 11  Total Number of T

## 2. Building The First Regression Model: Random Forest Regression

After cleaning the  data, we are now going to build our regressor since we are predicting indexs not classes. Here, considering possible missing data in hospitalization, the properties of random forest regressor will help us eliminate the impact of missing data. So we going to build a Rondom forest regressor.

First we will our elements using for predication and our desired predication data. Since we going to predict the trend of pandemic in New York city, it's the best for us to use "New Positives" as our prediction target. And we will use all the other elements except for the object element "County" as our training elements. The longtitude and the latitude elements could represents the geographical effect of the virus instead of "County" element.

In [0]:
y = datff.iloc[:,9].values
X = datff.iloc[:, [0,1,2,3,4,5,6,7,10,11,12,13]].values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 30)

Due to different length of indexs of each elements, we need to preprocess the scale of the data to make sure they are all normalized and easy to handled by our regressor.

In [0]:
# Preprocessing
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)
StandardScaler(copy=True, with_mean=True, with_std=True)
# Now apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

After preprocessing we need find the best parameters for the regressor, we are going to use the random grid function in sklearn to help with it. In the following part we will list all the parameters we want the algorithm to choose for us in ten randomly learned models. 

Other than this setting, we will set the regressor's critetion to "MAE" in order to eliminate any significant impact of anomalous datapoints to the learning algorithm.

In [0]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 100, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [0]:
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [1, 12, 23, 34, 45, 56, 67, 78, 89, 100, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [0]:
# Check for the best parameters of regressor
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator=rf, param_distributions = random_grid, cv=10)
rf_random.fit(X_train,y_train)
print(rf_random.best_params_)

{'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 56, 'bootstrap': True}


We have the best regressor model as the random grid function searched for us, now we will use this elements to build our regressor and train our model on training sets.

In [0]:
# Train the model
ranforreg = RandomForestRegressor(n_estimators=200, criterion = "mae", min_samples_leaf = 2, max_features = "auto", random_state=30, bootstrap=True, min_samples_split=5, max_depth = 56) 
ranforreg.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=56, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=30, verbose=0, warm_start=False)

After training the model, we could use the model to predict on our test sets to see its performance.

In [0]:
# Test the model and see how it fits the regression
predictions = ranforreg.predict(X_test)
print(r2_score(y_test, predictions))

0.9827466187908719


In [0]:
scores = cross_val_score(ranforreg, X_test, y_test, cv = 5)

In [0]:
scores.mean()  

0.9457228455817809

After testing, our model was graded by the R2 score of fitness 98.27%, comparing with the original target datapoints. For best evaluation of our model, we implement 5-fold cross validation on our model, and it wins a average score of 94.57% of our model.

## 3. Second Model : KNN Regressor

In order to check to make sure our datasets is usable for further prediction of the trend, we will build another regressor in order to check.

Here we are going to use the K-neighbourhood regressor in order to predict the trend in New York state by counties since I believe pandemic performance in the same area will probably have close features reflected by the datapoints.

We will still start with select parameters for the regressor. Here we will introduce grid search function in sklearn in order to help us with this problem.

In [0]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

In [0]:
grid_params = {
    'n_neighbors': [3, 9, 11, 19],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan','minkowski'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree'],
    'leaf_size': [10, 30, 50],
}
KNN = KNeighborsRegressor()
gs = GridSearchCV(KNN, grid_params, verbose = 1, cv = 5)

In [25]:
gs_result = gs.fit(X_train, y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1080 out of 1080 | elapsed:   21.5s finished


In [27]:
print(gs_result.best_params_)

{'algorithm': 'auto', 'leaf_size': 10, 'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}


After training the grid search CV using our training data, we find the best parameters of the algorithm. Now we will implement these paramters and train our model using the training sets.

In [31]:
KNN2 = KNeighborsRegressor(
    algorithm='auto', leaf_size=10, metric='euclidean',
    metric_params=None, n_jobs=None, n_neighbors=3, p=2,
    weights='distance')
KNN2.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=10, metric='euclidean',
                    metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                    weights='distance')

After training our KNN regressor, now we could predict the data with our test sets and compare with our test results with the orginal data. And then using the 5-fold cross validation to test the performance of the model.

In [33]:
prediction2 = KNN2.predict(X_test)
print(r2_score(y_test, prediction2))

0.9745914789642072


In [0]:
scores = cross_val_score(KNN2, X_test, y_test, cv = 5)

In [35]:
scores.mean()  

0.9333196671331014

We could see that our KNN regressor have a R2 score of fitness of 97.46% and a average performance score of 93.32%. It shows that KNN regressor is also a possible model to predict trend of pandemic in the future in New York State.

## Conclusion and Data Visualization

![20200506_104748.gif](https://i.loli.net/2020/05/06/7CibQYjNmK8Spve.gif)

The former GIF is a sorting image of weekly new positive cases in each county in New York states. We could possibly see a trend along the timeline.

After our data visualisation and testing of preditor models, we could possibly conclude that the pandemic trend in New York can be predicted using several known data and common statistics of the counties.